In [1]:
from nltk.corpus import reuters
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import LinearSVC
from sklearn.cross_validation import cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from pprint import pprint
from nltk.corpus import stopwords

# Data Collection Stats

In [ ]:
# List of documents
documents = reuters.fileids()
print("Documents: {}".format(len(documents)))

In [ ]:
train_docs = list(filter(lambda doc: doc.startswith("train"), documents))
print("Total train documents: {}".format(len(train_docs)))

test_docs = list(filter(lambda doc: doc.startswith("test"), documents))
print("Total test documents: {}".format(len(test_docs)))

In [ ]:
# List of categories 
categories = reuters.categories();
print("Number of categories: {}".format(len(categories)))

categories

In [ ]:
# Documents per category. TODO: Lambdas too complex?
category_distribution = sorted([[category, len(reuters.fileids(category))] for category in categories], 
                               key=lambda item:item[1], 
                               reverse=True)

category_distribution[:10]

In [ ]:
category_distribution[-10:]

In [ ]:
# Number of labels (different than number of documents...)
sum([distribution for category, distribution in category_distribution])

In [ ]:
# Documents with multiple labels
doc = 'training/9865'
print(reuters.raw(doc))

reuters.categories(doc)

In [2]:
# Function to show the feature weights of a document (to be explained later)
def feature_values(doc, representer):
    doc_representation = representer.transform([doc])
    features = representer.get_feature_names()
    return [(features[index], doc_representation[0, index]) for index in doc_representation.nonzero()[1]]

In [ ]:
# Tokenisation 
vectorizer = TfidfVectorizer()

# Learn and transform train documents
vectorised_train_documents = vectorizer.fit_transform(train_docs)
vectorised_test_documents = vectorizer.transform(test_docs)

# Transform multilabel labels
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform([reuters.categories(document) for document in train_docs])
test_labels = mlb.transform([reuters.categories(document) for document in test_docs])

# Classifier 
classifier = OneVsRestClassifier(LinearSVC())
classifier.fit(vectorised_train_documents, train_labels)
predictions = classifier.predict(vectorised_test_documents)

sum([sum(prediction) for prediction in predictions])

In [ ]:
y = mlb.fit_transform(all_labels)
    classifier = OneVsRestClassifier(LinearSVC())

    scores = cross_val_score(classifier, X, y=y, cv=10, scoring='f1_micro')
    print("Average F1: {}".format(np.mean(scores)))